Daniel M. Smith Flatiron School

### WebScraping The-numbers.com
Call Build URL string and input into getdfFromURL to download data fromt TheNumbers Query page.<br>
Can only select 100 per scrape means more queries
## Do Not run all or you will query many times

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import requests


In [4]:
def getdffromUrl(url):
    '''
    This getdffromUrl function returns a dataframe from the callung url. 

    :param url: The url to create a BeautifulSoup object in turn parsing the 
     report table and returning a dataframe.
    :type str: 
    
    :returns: A pandas dataframe of the report table from the-numbers
    :rtype: pandas.df
    '''
    
    html_page = requests.get(url) # Make a get request to retrieve the page
    soup = BeautifulSoup(html_page.content, 'html.parser') # Pass the page contents to beautiful soup for parsing

    table_container = soup.find('div', id="page_filling_chart")
    table_headers = table_container.findAll('th')
    table_rows = table_container.findAll('tr')
    return parse_table2(table_headers,table_rows)

In [5]:
#genre = All, 
#creative_type=All, Science Fiction,Dramatization,Factual,Historical Fiction, Kids Fiction, Multiple Creative Types, 
#   Contempory Fiction, Fantasy, Super Hero
#Distributor
#Prod Method
#'All/All/All/All/'
def buildstarturl(genre='All', creative_type='All', domestic_distributor='All',
                  prod_method='All' ):
    starturl = genre + '/' + creative_type + '/' + domestic_distributor + '/'\
    + prod_method + '/'
    return starturl

In [6]:
#rating-
#creativesourceidea
#Productioncountry
#ProductionCompany
#Keywords
#-(US)/All/All/All/All/
def buildmidurl(rating='All', creative_source='All', prod_country='All',
                  prod_company='All',keywords='All' ):
    midurl = rating + '-(US)/' + creative_source + '/' + prod_country + '/'\
            + prod_company + '/' + keywords + '/'
    return midurl

In [7]:
#Min Budget
#maxbudget
#min release year
#max release year
#min release month
#max release month
#min runtime
#max runtime
#minopening weekend
#max opening weekend
#.05/400/2010/2010/7/12/61/2020/.5/358
#/None/None/None/None/None/None
def buildendurl(minbudget='None',maxbudget='400', min_rel_yr='None', max_rel_yr='None',
                min_rel_mth='None', max_rel_mth='None', minruntime='0',
                maxruntime='2048', min_open_wknd='None', max_open_wknd='None'):
    
    endurl = minbudget + '/' + maxbudget + '/' + min_rel_yr + '/'\
    + max_rel_yr + '/' + min_rel_mth + '/' + max_rel_mth + '/'\
    + minruntime + '/' + maxruntime + '/' + min_open_wknd + '/' + max_open_wknd
    
    return endurl

In [26]:
# Builds the URL String with inputs on MPAA rating, Min Budget,MaxBudget, Start Year, End Year
def buildurl(rating, minbudget, maxbudget, startyear, endyear):
    '''
    This buildurl function returns a full url string to query the-numbers.com 
    site.

    :param rating: The MPAA rating of movies to query
    :type amount: 
    :param minbudget: The minumum production budget to query
    :type amount: str
    :param maxbudget: The minumum production budget to query
    :type amount: str
    :param startyear: The start year to query
    :type amount: str
    :param endyear: The end year to query
    :type amount: str

    :returns: A url calling the-numbers reporting page
    :rtype: str
    '''

    baseurl = 'https://www.the-numbers.com/movies/report/'
    #'All/All/All/All/'
    starturl = buildstarturl(genre='All', creative_type='All', 
                             domestic_distributor='All',
                             prod_method='All' )#'All/All/All/All/'
    
    midurl = buildmidurl(rating,creative_source='All', prod_country='All',
                  prod_company='All',keywords='All')#'-(US)/All/All/All/All/'
    
    endurl = buildendurl(minbudget=minbudget,maxbudget=maxbudget, 
                         min_rel_yr=startyear, max_rel_yr=endyear,
                                min_rel_mth='None', max_rel_mth='None', 
                         minruntime='0', maxruntime='2048', 
                         min_open_wknd='None', max_open_wknd='None')#'/None/None/None/None/None/None'
    
    params = '?show-release-date=On&show-release-year=Off'\
    +'&show-production-budget=On&show-opening-weekend-theaters=On'\
            +'&show-domestic-box-office=On'\
            +'&show-maximum-theaters=On'\
            +'&show-inflation-adjusted-domestic-box-office=On'\
            +'&show-theatrical-engagements=On'\
            +'&show-international-box-office=On'\
            +'&show-opening-weekend-revenue=On'\
            +'&show-worldwide-box-office=On'\
            +'&show-worldwide-release-date=On'\
            +'&show-worldwide-release-year=Off'\
            +'&show-theatrical-distributor=On'\
            +'&show-genre=On&show-source=On'\
            +'&show-production-method=On'\
            +'&show-creative-type=On'\
            +'&view-order-by=domestic-box-office'\
            +'&view-order-direction=desc'

    url = baseurl + starturl + midurl + endurl + params

    return url

In [32]:
# Final Parse table from the numbers returns a data frame
def parse_table2(table_headers, table_rows):
    '''
    This parse_table2 function returns a dataframe from the inputs of
    html table headers and html table rows. 

    :param table_headers: The table_headers of an html table 
    :type ResultSet: 
    :param table_rows: The table_rows of an html table 
    :type ResultSet: 
    
    :returns: A pandas dataframe of the report table from the-numbers
    :rtype: pandas.df
    '''
    headers = []
    movies = []

    # create list of headers
    for i in range(0, len(table_headers)):
        headers.append(table_headers[i].get_text())

    # create list of rows/datacells
    for i in range(1, len(table_rows)):
        tds = table_rows[i].findAll('td')
        movie_dict = {}
        for i in range(0, len(tds)):
            #print(table_headers[i].get_text() ,tds[i].get_text())
            movie_dict[table_headers[i].get_text()] = tds[i].get_text()
        movies.append(movie_dict)

    df = pd.DataFrame(movies)
    df.drop(columns=headers[0], inplace=True)

    return df

In [657]:
df_g_rated_2 = parse_table2(table_headers,table_rows)

In [658]:
df_g_rated_2

,Released,Title,Source
0,"Feb 11, 2011",Gnomeo and Juliet,Based on Play
1,"Feb 11, 2011",Justin Bieber: Never Say Never,Based on Real Life Events
2,"Jul 15, 2011",Winnie the Pooh,Based on Fiction Book/Short Story
3,"Jul 23, 2010",Ramona and Beezus,Based on Fiction Book/Short Story
4,"Feb 17, 2012",Kari gurashi no Arietti,Based on Fiction Book/Short Story
5,"Dec 13, 2011",George Balanchine's The Nutcracker,Based on Ballet
6,"Oct 21, 2011",The Mighty Macs,Based on Real Life Events
7,"Aug 29, 2012",The Oogieloves in the BIG Balloon Adv…,Original Screenplay
8,"Oct 4, 2011",La vÃ©ritable histoire du Chat BottÃ©,
9,"Dec 31, 2012",Zambezia,Original Screenplay


In [13]:
buildurl('G','2','65','2010','2018')

'https://www.the-numbers.com/movies/report/All/All/All/All/G-(US)/All/All/All/All/2/65/2010/2018/None/None/0/2048/None/None?show-release-date=On&show-release-year=On&show-production-budget=On&show-opening-weekend-theaters=On&show-domestic-box-office=On&show-maximum-theaters=On&show-inflation-adjusted-domestic-box-office=On&show-theatrical-engagements=On&show-international-box-office=On&show-opening-weekend-revenue=On&show-worldwide-box-office=On&show-worldwide-release-date=On&show-worldwide-release-year=On&show-theatrical-distributor=On&show-genre=On&show-source=On&show-production-method=On&show-creative-type=On&view-order-by=domestic-box-office&view-order-direction=desc'

In [33]:
df_G = getdffromUrl(buildurl('G','.05','400','2010','2019'))

In [31]:
df_PG_to2011

,Released,ReleasedWorldwide,Title,TheatricalDistributor,Genre,Source,ProductionMethod,CreativeType,ProductionBudget,Opening WeekendTheaters,MaximumTheaters,TheatricalEngagements,Opening WeekendRevenue,DomesticBox Office,Infl. Adj. Dom.Box Office,InternationalBox Office,WorldwideBox Office
0,"Mar 5, 2010","Mar 4, 2010",Alice in Wonderland,Walt Disney,Adventure,Based on Fiction Book/Short Story,Animation/Live Action,Fantasy,"$200,000,000","3,728","3,739","27,013","$116,101,023","$334,191,110","$387,983,589","$691,300,000","$1,025,491,110"
1,"Jul 9, 2010","Jul 9, 2010",Despicable Me,Universal,Adventure,Original Screenplay,Digital Animation,Kids Fiction,"$69,000,000","3,476","3,602","33,507","$56,397,125","$251,513,985","$291,996,673","$291,950,588","$543,464,573"
2,"May 21, 2010","May 21, 2010",Shrek Forever After,Paramount Pictures,Adventure,Based on Fiction Book/Short Story,Digital Animation,Kids Fiction,"$165,000,000","4,359","4,386","25,897","$70,838,207","$238,736,787","$277,164,636","$517,507,886","$756,244,673"
3,"Mar 26, 2010","Mar 26, 2010",How to Train Your Dragon,Paramount Pictures,Adventure,Based on Fiction Book/Short Story,Digital Animation,Fantasy,"$165,000,000","4,055","4,060","33,192","$43,732,319","$217,581,232","$252,603,809","$277,289,760","$494,870,992"
4,"Nov 24, 2010","Nov 24, 2010",Tangled,Walt Disney,Musical,Based on Folk Tale/Legend/Fairytale,Digital Animation,Kids Fiction,"$260,000,000","3,603","3,603","32,392","$48,767,052","$200,821,936","$232,953,630","$384,752,381","$585,574,317"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,"May 1, 2010","May 1, 2010",My Girlfriend's Boyfriend,,Romantic Comedy,,Live Action,,"$1,400,000",0,0,0,$0,$0,$0,$0,$0
79,"Jun 3, 2011","Jun 3, 2011",The Lion of Judah,Rocky Mountain Pictures,Adventure,Based on Religious Text,Digital Animation,Kids Fiction,"$1,500,000",0,0,0,$0,$0,$0,$0,$0
80,"Dec 31, 2010","Dec 31, 2010",Konferenz der Tiere,,Adventure,Original Screenplay,Digital Animation,Kids Fiction,"$30,000,000",0,0,0,$0,$0,$0,"$53,048,539","$53,048,539"
81,"Mar 15, 2011","Mar 15, 2011",A Shine of Rainbows,,Drama,,,,"$6,000,000",0,0,0,$0,$0,$0,$0,$0


In [34]:
df_PG_to2011 = getdffromUrl(buildurl('PG','.05','400','2010','2011'))

In [35]:
df_PG_to2012  = getdffromUrl(buildurl('PG','.05','400','2012','2012'))

In [36]:
df_PG_to2013  = getdffromUrl(buildurl('PG','.05','400','2013','2013'))

In [37]:
df_PG_to2014  = getdffromUrl(buildurl('PG','.05','400','2014','2014'))

In [38]:
df_PG_to2015  = getdffromUrl(buildurl('PG','.05','400','2015','2015'))

In [39]:
df_PG_to2016  = getdffromUrl(buildurl('PG','.05','400','2016','2016'))

In [40]:
df_PG_to2017  = getdffromUrl(buildurl('PG','.05','400','2017','2017'))

In [41]:
df_PG_to2018  = getdffromUrl(buildurl('PG','.05','400','2018','2018'))

In [42]:
df_PG_to2019  = getdffromUrl(buildurl('PG','.05','400','2019','2019'))

In [43]:
df_PG_to2020  = getdffromUrl(buildurl('PG','.05','400','2020','2020'))

In [44]:
df_PG_to2021  = getdffromUrl(buildurl('PG','.05','400','2014','2014'))

In [45]:
df_PG13_2010  = getdffromUrl(buildurl('PG-13','.05','400','2010','2010'))
len(df_PG13_2010)

71

In [46]:
df_PG13_2011  = getdffromUrl(buildurl('PG-13','.05','400','2011','2011'))
len(df_PG13_2011)

89

In [47]:
df_PG13_2012  = getdffromUrl(buildurl('PG-13','.05','400','2012','2012'))
len(df_PG13_2012)

81

In [48]:
df_PG13_2013  = getdffromUrl(buildurl('PG-13','.05','400','2013','2013'))

In [49]:
df_PG13_2014  = getdffromUrl(buildurl('PG-13','.05','400','2014','2014'))

In [50]:
df_PG13_2015  = getdffromUrl(buildurl('PG-13','.05','400','2015','2015'))

In [51]:
df_PG13_2016  = getdffromUrl(buildurl('PG-13','.05','400','2016','2016'))

In [52]:
df_PG13_2017  = getdffromUrl(buildurl('PG-13','.05','400','2017','2017'))

In [53]:
df_PG13_2018  = getdffromUrl(buildurl('PG-13','.05','400','2018','2018'))

In [54]:
df_PG13_2019  = getdffromUrl(buildurl('PG-13','.05','400','2019','2019'))

In [55]:
df_PG13_2020  = getdffromUrl(buildurl('PG-13','.05','400','2020','2020'))

In [56]:
df_PG13_2021  = getdffromUrl(buildurl('PG-13','.05','400','2021','2021'))

In [57]:
df_R_2020  = getdffromUrl(buildurl('R','.05','400','2020','2020'))

In [58]:
df_R_2021  = getdffromUrl(buildurl('R','.05','400','2021','2021'))

In [59]:
df_R_2019  = getdffromUrl(buildurl('R','.05','400','2019','2019'))

In [60]:
df_R_2018  = getdffromUrl(buildurl('R','.05','400','2018','2018'))

In [61]:
df_R_2017  = getdffromUrl(buildurl('R','.05','400','2017','2017'))

In [83]:
df_R_2016_1  = getdffromUrl(buildurl('R','.05','30','2016','2016'))

In [82]:
df_R_2016_2  = getdffromUrl(buildurl('R','30.01','200','2016','2016'))

In [80]:
df_R_2015_1  = getdffromUrl(buildurl('R','.05','30','2015','2015'))

In [79]:
df_R_2015_2  = getdffromUrl(buildurl('R','30.01','400','2015','2015'))
len(df_R_2015_2)

23

In [78]:
df_R_2014_2  = getdffromUrl(buildurl('R','30.01','400','2014','2014'))

In [77]:
df_R_2014_1  = getdffromUrl(buildurl('R','.05','30','2014','2014'))

In [76]:
df_R_2013_1  = getdffromUrl(buildurl('R','.05','30','2013','2013'))
len(df_R_2013_1)

81

In [75]:
df_R_2013_2  = getdffromUrl(buildurl('R','30.01','400','2013','2013'))
len(df_R_2013_2)

24

In [74]:
df_R_2012_1  = getdffromUrl(buildurl('R','.05','30','2012','2012'))
len(df_R_2012_1)

75

In [73]:
df_R_2012_2  = getdffromUrl(buildurl('R','30.01','400','2012','2012'))
len(df_R_2012_2)

29

In [71]:
df_R_2011_1  = getdffromUrl(buildurl('R','.05','30','2011','2011'))
len(df_R_2011_1)

85

In [64]:
df_R_2011_2  = getdffromUrl(buildurl('R','30.01','400','2011','2011'))
len(df_R_2011_2)

23

In [65]:
df_R_2010_1  = getdffromUrl(buildurl('R','.05','30','2010','2010'))
len(df_R_2010_1)

99

In [66]:
df_R_2010_2  = getdffromUrl(buildurl('R','30.01','400','2010','2010'))
len(df_R_2010_2)

22

In [67]:
df_all = pd.DataFrame()

In [68]:
df_all = df_all.append(df_G,ignore_index=True)

In [84]:
df_all = df_all.append(df_PG_to2011,ignore_index=True)
df_all = df_all.append(df_PG_to2012,ignore_index=True)
df_all = df_all.append(df_PG_to2013,ignore_index=True)
df_all = df_all.append(df_PG_to2014,ignore_index=True)
df_all = df_all.append(df_PG_to2015,ignore_index=True)
df_all = df_all.append(df_PG_to2016,ignore_index=True)
df_all = df_all.append(df_PG_to2017,ignore_index=True)
df_all = df_all.append(df_PG_to2018,ignore_index=True)
df_all = df_all.append(df_PG_to2019,ignore_index=True)
df_all = df_all.append(df_PG_to2020,ignore_index=True)
df_all = df_all.append(df_PG_to2021,ignore_index=True)

df_all = df_all.append(df_PG13_2010,ignore_index=True)
df_all = df_all.append(df_PG13_2011,ignore_index=True)
df_all = df_all.append(df_PG13_2012,ignore_index=True)
df_all = df_all.append(df_PG13_2013,ignore_index=True)
df_all = df_all.append(df_PG13_2014,ignore_index=True)
df_all = df_all.append(df_PG13_2015,ignore_index=True)
df_all = df_all.append(df_PG13_2016,ignore_index=True)
df_all = df_all.append(df_PG13_2017,ignore_index=True)
df_all = df_all.append(df_PG13_2018,ignore_index=True)
df_all = df_all.append(df_PG13_2019,ignore_index=True)
df_all = df_all.append(df_PG13_2020,ignore_index=True)
df_all = df_all.append(df_PG13_2021,ignore_index=True)
    
df_all = df_all.append(df_R_2010_1,ignore_index=True)
df_all = df_all.append(df_R_2010_2,ignore_index=True)
df_all = df_all.append(df_R_2011_1,ignore_index=True)
df_all = df_all.append(df_R_2011_2,ignore_index=True)
df_all = df_all.append(df_R_2012_1,ignore_index=True)
df_all = df_all.append(df_R_2012_2,ignore_index=True)
df_all = df_all.append(df_R_2013_1,ignore_index=True)
df_all = df_all.append(df_R_2013_2,ignore_index=True)
df_all = df_all.append(df_R_2014_1,ignore_index=True)
df_all = df_all.append(df_R_2014_2,ignore_index=True)
df_all = df_all.append(df_R_2015_1,ignore_index=True)
df_all = df_all.append(df_R_2015_2,ignore_index=True)
df_all = df_all.append(df_R_2016_1,ignore_index=True)
df_all = df_all.append(df_R_2016_2,ignore_index=True)
df_all = df_all.append(df_R_2017,ignore_index=True)
df_all = df_all.append(df_R_2018,ignore_index=True)
df_all = df_all.append(df_R_2019,ignore_index=True)
df_all = df_all.append(df_R_2020,ignore_index=True)
df_all = df_all.append(df_R_2021,ignore_index=True)

In [85]:
df_all.to_csv('./data/tn_moviesource2.csv', mode='a', header=True, encoding='utf-8')

In [81]:
tn_df

,Unnamed: 0,Released,ReleasedWorldwide,Title,TheatricalDistributor,Genre,Source,ProductionMethod,CreativeType,ProductionBudget,Opening WeekendTheaters,MaximumTheaters,TheatricalEngagements,Opening WeekendRevenue,DomesticBox Office,Infl. Adj. Dom.Box Office,InternationalBox Office,WorldwideBox Office
0,0,2019,2019,Toy Story 4,Walt Disney,Adventure,Original Screenplay,Digital Animation,Kids Fiction,"$200,000,000","4,575","4,575","38,194","$120,908,065","$434,038,008","$434,038,008","$639,042,321","$1,073,080,329"
1,1,2010,2010,Toy Story 3,Walt Disney,Adventure,Original Screenplay,Digital Animation,Kids Fiction,"$200,000,000","4,028","4,028","33,699","$110,307,189","$415,004,880","$481,805,411","$653,874,642","$1,068,879,522"
2,2,2013,2013,Monsters University,Walt Disney,Adventure,Original Screenplay,Digital Animation,Kids Fiction,"$200,000,000","4,004","4,004","24,656","$82,429,469","$268,488,329","$302,503,449","$474,967,481","$743,455,810"
3,3,2011,2011,Cars 2,Walt Disney,Adventure,Original Screenplay,Digital Animation,Kids Fiction,"$200,000,000","4,115","4,115","25,790","$66,135,507","$191,450,875","$221,146,280","$368,704,508","$560,155,383"
4,4,2017,2017,Cars 3,Walt Disney,Adventure,Original Screenplay,Digital Animation,Kids Fiction,"$175,000,000","4,256","4,256","24,083","$53,688,680","$152,901,115","$156,139,821","$230,640,254","$383,541,369"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2833,2833,2021,2021,Our Friend,Roadside Attractions,Drama,Based on Factual Book/Article,Live Action,Dramatization,"$10,000,000",543,818,"2,405","$250,000","$657,351","$657,351","$56,744","$714,095"
2834,2834,2021,2021,Malcolm & Marie,NaN,Drama,Original Screenplay,Live Action,Contemporary Fiction,"$2,500,000",0,0,0,$0,$0,$0,$0,$0
2835,2835,2021,2021,Army of the Dead,Netflix,Action,Original Screenplay,Live Action,Science Fiction,"$90,000,000",0,0,0,$0,$0,$0,$0,$0
2836,2836,2021,2021,Midnight in the Switchgrass,Lionsgate,Thriller/Suspense,Based on Real Life Events,Live Action,Dramatization,"$15,000,000",0,0,0,$0,$0,$0,"$36,248","$36,248"


In [576]:
df_r_rated_2018.to_csv('../data/tn_mpaa_ratings.csv', mode='a', header=False)

#The Numbers HOLDIng Cell for Initial URLS
G  2010-2018 'https://www.the-numbers.com/movies/report/All/All/All/All/G-(US)/All/All/All/All/2/65/2010/2019/None/None/None/None/None/None?show-release-date=On&view-order-by=domestic-box-office&view-order-direction=desc&show-production-budget=On&show-worldwide-box-office=On'
PG 2010-2014 'https://www.the-numbers.com/movies/report/All/All/All/All/PG-(US)/All/All/All/All/2/65/2010/2014/None/None/None/None/None/None?show-release-date=On&view-order-by=domestic-box-office&view-order-direction=desc&show-production-budget=On&show-worldwide-box-office=On'
PG 2015-2019 'https://www.the-numbers.com/movies/report/All/All/All/All/PG-(US)/All/All/All/All/2/65/2015/2019/None/None/None/None/None/None?show-release-date=On&view-order-by=domestic-box-office&view-order-direction=desc&show-production-budget=On&show-worldwide-box-office=On'

PG-13 2010   'https://www.the-numbers.com/movies/report/All/All/All/All/PG-13-(US)/All/All/All/All/2/65/2010/2010/None/None/None/None/None/None?show-release-date=On&view-order-by=domestic-box-office&view-order-direction=desc&show-production-budget=On&show-worldwide-box-office=On'
PG-13 2011   https://www.the-numbers.com/movies/report/All/All/All/All/PG-13-(US)/All/All/All/All/2/65/2011/2011/None/None/None/None/None/None?show-release-date=On&view-order-by=domestic-box-office&view-order-direction=desc&show-production-budget=On&show-domestic-box-office=On&show-international-box-office=On&show-worldwide-box-office=On&show-theatrical-distributor=On&show-genre=On&show-source=On
PG-13 2012   https://www.the-numbers.com/movies/report/All/All/All/All/PG-13-(US)/All/All/All/All/2/65/2012/2012/None/None/None/None/None/None?show-release-date=On&view-order-by=domestic-box-office&view-order-direction=desc&show-production-budget=On&show-domestic-box-office=On&show-international-box-office=On&show-worldwide-box-office=On&show-theatrical-distributor=On&show-genre=On&show-source=On
PG-13 2013
PG-13 2014
PG-13 2015
PG-13 2016
PG-13 2017
PG-13 2018

R 2010 2m-30 m  https://www.the-numbers.com/movies/report/All/All/All/All/R-(US)/All/All/All/All/2/30/2010/2010/None/None/None/None/None/None?show-release-date=On&view-order-by=domestic-box-office&view-order-direction=desc&show-production-budget=On&show-domestic-box-office=On&show-international-box-office=On&show-worldwide-box-office=On&show-theatrical-distributor=On&show-genre=On&show-source=On
R 2010 31m-65m  https://www.the-numbers.com/movies/report/All/All/All/All/R-(US)/All/All/All/All/31/65/2010/2010/None/None/None/None/None/None?show-release-date=On&view-order-by=domestic-box-office&view-order-direction=desc&show-production-budget=On&show-domestic-box-office=On&show-international-box-office=On&show-worldwide-box-office=On&show-theatrical-distributor=On&show-genre=On&show-source=On
R 2011
R 2012
R 2013
R 2014
R 2015
R 2016
R 2017
R 2018 'https://www.the-numbers.com/movies/report/All/All/All/All/R-(US)/All/All/All/All/2/65/2018/2018/None/None/None/None/None/None?show-release-date=On&view-order-by=domestic-box-office&view-order-direction=desc&show-production-budget=On&show-worldwide-box-office=On'



R 2018 https://www.the-numbers.com/movies/report/All/All/All/All/R-(US)/All/All/All/All/2/65/2018/2018/None/None/None/None/None/None?show-release-date=On&view-order-by=domestic-box-office&view-order-direction=desc&show-production-budget=On&show-domestic-box-office=On&show-international-box-office=On&show-worldwide-box-office=On&show-theatrical-distributor=On&show-genre=On&show-source=On

url='https://www.the-numbers.com/movies/report/All/All/All/All/G-(US)/All/All/All/All/2/65/2010/2019/None/None/None/None/None/None?show-release-date=On&view-order-by=domestic-box-office&view-order-direction=desc&show-source=On'
html_page = requests.get(url) # Make a get request to retrieve the page
soup = BeautifulSoup(html_page.content, 'html.parser') # Pass the page contents to beautiful soup for parsing

table_container = soup.find('div', id="page_filling_chart")
table_headers = table_container.findAll('th')
table_rows = table_container.findAll('tr')